# Ad-Gen-Pipeline Validation Notebook

This notebook validates each step of the advertisement image generation pipeline:
1. **Object Matting**: Remove background from product image
2. **Background Generation**: Create atmospheric background
3. **Spatial Analysis**: Find optimal placement location
4. **Object Synthesis**: Blend object into background with natural lighting

## Cell 1: Environment Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

# Try to import helper_dev_utils, fallback to standard logging if not available
try:
    from helper_dev_utils import get_auto_logger
    logger = get_auto_logger()
except ImportError:
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

In [ ]:
# Import libraries
import sys
from pathlib import Path
import torch
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Import our modules
from src import (
    flush_gpu,
    load_image,
    save_image,
    ObjectMatting,
    BackgroundGenerator,
    SpatialAnalyzer,
    ObjectSynthesizer
)
from src.utils import get_device, print_gpu_memory

# Check GPU availability
device = get_device()
print_gpu_memory()

# Helper function to display images
def show_images(images, titles=None, figsize=(15, 5)):
    """Display multiple images side by side."""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    
    for i, (img, ax) in enumerate(zip(images, axes)):
        ax.imshow(img)
        ax.axis('off')
        if titles and i < len(titles):
            ax.set_title(titles[i])
    
    plt.tight_layout()
    plt.show()

logger.debug("\n✅ Environment setup complete!")

## Cell 2: Input Data Definition (User Scenario)

In [ ]:
# ==================================================
# USER CONFIGURATION: Modify these parameters
# ==================================================

# Input product image path
INPUT_IMAGE_PATH = "../image.png"  # 노가리 이미지

# Scenario: 노가리를 술안주로 제공하는 포장마차/선술집 장면
PROMPT_SCENARIO = (
    "A photorealistic shot of dried pollack fish (nogari) as a Korean bar snack "
    "on a wooden table in a traditional Korean bar (pojangmacha), "
    "warm ambient lighting, cozy atmosphere, shallow depth of field, "
    "natural food photography, cinematic style"
)

# Background-specific prompt (remove product mentions)
PROMPT_BACKGROUND = (
    "An empty wooden table in a cozy Korean traditional bar (pojangmacha) "
    "with soft warm lighting, empty space in center for food placement, "
    "blurred background showing bar atmosphere with bokeh effect, "
    "shallow depth of field, warm tones, no food on table"
)

# Spatial query for object placement
SPATIAL_QUERY = (
    "Find the flat surface on the table center where I can place dried fish snacks. "
    "Return the bounding box coordinates for the empty area."
)

# Generation parameters
IMAGE_SIZE = (1024, 1024)  # (width, height)
SEED = 42  # For reproducibility
IP_ADAPTER_SCALE = 0.8  # 0.7=natural, 1.0=preserve original

# Output directory
OUTPUT_DIR = Path("../outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

logger.debug("📋 Configuration:")
print(f"   Input: {INPUT_IMAGE_PATH}")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Image size: {IMAGE_SIZE}")
print(f"   Seed: {SEED}")
print(f"\n💭 Scenario: {PROMPT_SCENARIO}")

## Cell 3: Step 1 - Object Matting (Background Removal)

Remove the background from the product image to create a clean reference.

In [ ]:
logger.debug("🔄 Step 1: Object Matting (Background Removal)\n")
logger.debug("=" * 60)

# Load original image
original_image = load_image(INPUT_IMAGE_PATH)
print(f"\nOriginal image loaded: {original_image.size}")

# Initialize ObjectMatting
matting = ObjectMatting()

# Remove background
clean_ref_image = matting.remove_background(
    INPUT_IMAGE_PATH,
    return_rgba=True
)

# Save result
output_path = OUTPUT_DIR / "step1_clean_reference.png"
save_image(clean_ref_image, output_path)

# Display results
logger.debug("\n📊 Validation Point: Is the background cleanly removed?")
show_images(
    [original_image, clean_ref_image],
    titles=["Original Product", "Background Removed (RGBA)"],
    figsize=(12, 6)
)

# Memory cleanup
del matting
flush_gpu()
print_gpu_memory()
logger.debug("\n✅ Step 1 complete!\n")

## Cell 4: Step 2 - Background Generation

Generate an atmospheric background without the product.

In [ ]:
logger.debug("🔄 Step 2: Background Generation\n")
logger.debug("=" * 60)

# Initialize BackgroundGenerator
bg_generator = BackgroundGenerator()

# Generate background
bg_image = bg_generator.generate_background(
    prompt=PROMPT_BACKGROUND,
    width=IMAGE_SIZE[0],
    height=IMAGE_SIZE[1],
    num_inference_steps=28,
    guidance_scale=3.5,
    seed=SEED
)

# Save result
output_path = OUTPUT_DIR / "step2_background.png"
save_image(bg_image, output_path)

# Display result
logger.debug("\n📊 Validation Point: Does the background have space for object placement?")
show_images(
    [bg_image],
    titles=["Generated Background"],
    figsize=(8, 8)
)

# Memory cleanup
del bg_generator
flush_gpu()
print_gpu_memory()
logger.debug("\n✅ Step 2 complete!\n")

## Cell 5: Step 3 - Spatial Analysis (Object Placement)

Analyze the background to find the optimal location for object placement.

In [ ]:
logger.debug("🔄 Step 3: Spatial Analysis (Object Placement)\n")
logger.debug("=" * 60)

# Initialize SpatialAnalyzer
analyzer = SpatialAnalyzer()

# Detect optimal surface
detection_result = analyzer.detect_surface(
    image=bg_image,
    query=SPATIAL_QUERY
)

bbox = detection_result['bbox']
print(f"\n📍 Detected bounding box: {bbox}")
print(f"   Model response: {detection_result['text'][:150]}...")

# Create mask
mask_image = analyzer.create_mask(
    image_size=detection_result['image_size'],
    bbox=bbox
)

# Save mask
output_path = OUTPUT_DIR / "step3_mask.png"
save_image(mask_image, output_path)

# Visualize bbox on background
bg_with_bbox = analyzer.visualize_bbox(
    image=bg_image,
    bbox=bbox,
    color='red',
    width=5
)

# Display results
logger.debug("\n📊 Validation Point: Is the mask positioned appropriately?")
show_images(
    [bg_image, bg_with_bbox, mask_image],
    titles=["Background", "With BBox", "Binary Mask"],
    figsize=(18, 6)
)

# Memory cleanup
del analyzer
flush_gpu()
print_gpu_memory()
logger.debug("\n✅ Step 3 complete!\n")

## Cell 6: Step 4 - Object Synthesis (Final Composition)

Synthesize the clean object into the background with natural lighting and shadows.

In [ ]:
logger.debug("🔄 Step 4: Object Synthesis (Final Composition)\n")
logger.debug("=" * 60)

# Initialize ObjectSynthesizer
synthesizer = ObjectSynthesizer()

# ===================================================================
# IMPORTANT: FluxFillPipeline does NOT support IP-Adapter!
# ===================================================================
# Option 1: Single Pipeline (텍스트만, 메모리 효율적 ~11GB)
#   - use_two_stage=False
#   - 참조 이미지는 무시되고 텍스트 프롬프트만 사용
#   - 메모리: ~11GB (FluxFillPipeline만 로드)
#
# Option 2: Two-Stage Pipeline (IP-Adapter 사용, 메모리 많이 필요 ~22GB+)
#   - use_two_stage=True
#   - FluxPipeline(IP-Adapter) → FluxFillPipeline 순차 실행
#   - 참조 이미지의 시각적 특징을 반영
#   - 메모리: ~22GB+ (모델 2개, 순차 로드/언로드)
# ===================================================================

# Choose one option:
USE_TWO_STAGE = False  # True: IP-Adapter 사용 (메모리 많이 필요), False: 텍스트만 (메모리 효율적)

if USE_TWO_STAGE:
    logger.debug("⚠️  2단계 파이프라인 실행 - IP-Adapter 사용 (메모리 ~22GB+ 필요)")
else:
    logger.debug("✅ 단일 파이프라인 실행 - 텍스트만 사용 (메모리 ~11GB, 효율적)")
    logger.debug("⚠️  참조 이미지는 무시되고 텍스트 프롬프트만 사용됩니다")

# Synthesize object into background
final_image = synthesizer.fill_in_object(
    background=bg_image,
    mask=mask_image,
    reference=clean_ref_image,
    prompt=PROMPT_SCENARIO,
    num_inference_steps=28,
    guidance_scale=3.5,
    ip_adapter_scale=IP_ADAPTER_SCALE,
    seed=SEED,
    use_two_stage=USE_TWO_STAGE
)

# Save final result
output_path = OUTPUT_DIR / "step4_final_result.png"
save_image(final_image, output_path)

# Display final comparison
logger.debug("\n📊 Validation Point: Is the object naturally blended with proper lighting/shadows?")
show_images(
    [clean_ref_image, bg_image, mask_image, final_image],
    titles=["Clean Reference", "Background", "Mask", "Final Result"],
    figsize=(20, 5)
)

# Show before/after comparison
logger.debug("\n🔍 Before/After Comparison:")
show_images(
    [bg_image, final_image],
    titles=["Background Only", "With Product"],
    figsize=(16, 8)
)

# Memory cleanup
del synthesizer
flush_gpu()
print_gpu_memory()
logger.debug("\n✅ Step 4 complete!\n")